In [52]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
     

In [53]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [54]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [55]:
from google.colab import files
uploaded=files.upload()

Saving employees.csv to employees (2).csv


In [56]:
df=spark.read.format("csv").load("employees.csv")
df.show()

+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|       _c0|   _c1|       _c2|            _c3|   _c4|    _c5|              _c6|                 _c7|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|                Team|
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|           Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|                null|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|             Finance|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|             Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|     Client Services|
|    Dennis|  Male| 4/18/1987|        1:35 AM|115163| 10.125|            false|            

In [57]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema=StructType([\
                  StructField("First Name",IntegerType(),True),\
                  StructField("Gender",StringType(),True),\
                  StructField("Start Date",StringType(),True),\
                  StructField("Last Login Time", StringType(),True),\
                  StructField("Salary",IntegerType(),True),\
                  StructField("Bonus%", IntegerType(),True),\
                  StructField("Senior Management",StringType(),True),\
                  StructField("Team",StringType(),True)\
                   ])

In [58]:
df=spark.read.csv('employees.csv',schema=schema)
df.show()

+----------+------+----------+---------------+------+------+-----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus%|Senior Management|                Team|
+----------+------+----------+---------------+------+------+-----------------+--------------------+
|      null|Gender|Start Date|Last Login Time|  null|  null|Senior Management|                Team|
|      null|  Male|  8/6/1993|       12:42 PM| 97308|  null|             true|           Marketing|
|      null|  Male| 3/31/1996|        6:53 AM| 61933|  null|             true|                null|
|      null|Female| 4/23/1993|       11:17 AM|130590|  null|            false|             Finance|
|      null|  Male|  3/4/2005|        1:00 PM|138705|  null|             true|             Finance|
|      null|  Male| 1/24/1998|        4:47 PM|101004|  null|             true|     Client Services|
|      null|  Male| 4/18/1987|        1:35 AM|115163|  null|            false|               Legal|


In [59]:
df=spark.read.option("header","true").csv("employees.csv")
df.show(10,False)

+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|Team                |
+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|Douglas   |Male  |8/6/1993  |12:42 PM       |97308 |6.945  |true             |Marketing           |
|Thomas    |Male  |3/31/1996 |6:53 AM        |61933 |4.17   |true             |null                |
|Maria     |Female|4/23/1993 |11:17 AM       |130590|11.858 |false            |Finance             |
|Jerry     |Male  |3/4/2005  |1:00 PM        |138705|9.34   |true             |Finance             |
|Larry     |Male  |1/24/1998 |4:47 PM        |101004|1.389  |true             |Client Services     |
|Dennis    |Male  |4/18/1987 |1:35 AM        |115163|10.125 |false            |Legal               |
|Ruby      |Female|8/17/1987 |4:20 PM        |65476 |10.012 |true             |Product     

In [60]:
df.show()

+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|                Team|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|           Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|                null|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|             Finance|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|             Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|     Client Services|
|    Dennis|  Male| 4/18/1987|        1:35 AM|115163| 10.125|            false|               Legal|
|      Ruby|Female| 8/17/1987|        4:20 PM| 65476| 10.012|             true|            

In [61]:
import pyspark 
from pyspark.sql.functions import split
split_col = pyspark.sql.functions.split(df['Start Date'], '/')
df2 = df.select("First Name","Gender","Start Date","Last Login Time","Salary","Bonus %","Senior Management","Team", split_col.getItem(0).alias('Month'),split_col.getItem(1).alias('Day'),split_col.getItem(2).alias('Year'))   
df2.show(truncate=False)

+----------+------+----------+---------------+------+-------+-----------------+--------------------+-----+---+----+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|Team                |Month|Day|Year|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+-----+---+----+
|Douglas   |Male  |8/6/1993  |12:42 PM       |97308 |6.945  |true             |Marketing           |8    |6  |1993|
|Thomas    |Male  |3/31/1996 |6:53 AM        |61933 |4.17   |true             |null                |3    |31 |1996|
|Maria     |Female|4/23/1993 |11:17 AM       |130590|11.858 |false            |Finance             |4    |23 |1993|
|Jerry     |Male  |3/4/2005  |1:00 PM        |138705|9.34   |true             |Finance             |3    |4  |2005|
|Larry     |Male  |1/24/1998 |4:47 PM        |101004|1.389  |true             |Client Services     |1    |24 |1998|
|Dennis    |Male  |4/18/1987 |1:35 AM        |115163|10.125 |false      

In [62]:
df1 = df.withColumn('Month', split(df['Start Date'], '/').getItem(0)) \
       .withColumn('Day', split(df['Start Date'], '/').getItem(1)) \
       .withColumn('Year', split(df['Start Date'], '/').getItem(2))
df1.show(truncate=False)


+----------+------+----------+---------------+------+-------+-----------------+--------------------+-----+---+----+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|Team                |Month|Day|Year|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+-----+---+----+
|Douglas   |Male  |8/6/1993  |12:42 PM       |97308 |6.945  |true             |Marketing           |8    |6  |1993|
|Thomas    |Male  |3/31/1996 |6:53 AM        |61933 |4.17   |true             |null                |3    |31 |1996|
|Maria     |Female|4/23/1993 |11:17 AM       |130590|11.858 |false            |Finance             |4    |23 |1993|
|Jerry     |Male  |3/4/2005  |1:00 PM        |138705|9.34   |true             |Finance             |3    |4  |2005|
|Larry     |Male  |1/24/1998 |4:47 PM        |101004|1.389  |true             |Client Services     |1    |24 |1998|
|Dennis    |Male  |4/18/1987 |1:35 AM        |115163|10.125 |false      

In [63]:
#df3 = df1.withColumn("Start Date", df.Start Date.strip())
'''from pyspark.sql.functions import trim
df3 = df1.withColumn("Start Date", trim())'''
df3=df1.select('Month','Day','Year')
df3.show()

+-----+---+----+
|Month|Day|Year|
+-----+---+----+
|    8|  6|1993|
|    3| 31|1996|
|    4| 23|1993|
|    3|  4|2005|
|    1| 24|1998|
|    4| 18|1987|
|    8| 17|1987|
|    7| 20|2015|
|   11| 22|2005|
|    8|  8|2002|
|    8| 12|1980|
|   10| 26|1997|
|   12|  1|1980|
|    1| 27|2008|
|    1| 14|1999|
|    6|  5|2016|
|    9| 21|2010|
|   12|  7|1986|
|   10| 23|1981|
|    7| 22|2010|
+-----+---+----+
only showing top 20 rows



In [64]:
df4=df1.select('First Name','Gender','Start Date')
df4.show()

+----------+------+----------+
|First Name|Gender|Start Date|
+----------+------+----------+
|   Douglas|  Male|  8/6/1993|
|    Thomas|  Male| 3/31/1996|
|     Maria|Female| 4/23/1993|
|     Jerry|  Male|  3/4/2005|
|     Larry|  Male| 1/24/1998|
|    Dennis|  Male| 4/18/1987|
|      Ruby|Female| 8/17/1987|
|      null|Female| 7/20/2015|
|    Angela|Female|11/22/2005|
|   Frances|Female|  8/8/2002|
|    Louise|Female| 8/12/1980|
|     Julie|Female|10/26/1997|
|   Brandon|  Male| 12/1/1980|
|      Gary|  Male| 1/27/2008|
|  Kimberly|Female| 1/14/1999|
|   Lillian|Female|  6/5/2016|
|    Jeremy|  Male| 9/21/2010|
|     Shawn|  Male| 12/7/1986|
|     Diana|Female|10/23/1981|
|     Donna|Female| 7/22/2010|
+----------+------+----------+
only showing top 20 rows



In [65]:
df5=df1.select('Last Login Time','Salary','Bonus %','Senior Management','Team')
df5.show()

+---------------+------+-------+-----------------+--------------------+
|Last Login Time|Salary|Bonus %|Senior Management|                Team|
+---------------+------+-------+-----------------+--------------------+
|       12:42 PM| 97308|  6.945|             true|           Marketing|
|        6:53 AM| 61933|   4.17|             true|                null|
|       11:17 AM|130590| 11.858|            false|             Finance|
|        1:00 PM|138705|   9.34|             true|             Finance|
|        4:47 PM|101004|  1.389|             true|     Client Services|
|        1:35 AM|115163| 10.125|            false|               Legal|
|        4:20 PM| 65476| 10.012|             true|             Product|
|       10:43 AM| 45906| 11.598|             null|             Finance|
|        6:29 AM| 95570| 18.523|             true|         Engineering|
|        6:51 AM|139852|  7.524|             true|Business Development|
|        9:01 AM| 63241| 15.132|             true|              

In [74]:
#df4.join(df3,df5['Last Login Time']== df3['Month']).show()
'''import pandas as pd
df4.merge(df3,on='Month').merge(df5,on='Last Login Time')'''
#dataframe1=df4.unionByName(df3)
#dataframe1.show()
res = df4.unionByName(df3, allowMissingColumns=True)
res.show()

+----------+------+----------+-----+----+----+
|First Name|Gender|Start Date|Month| Day|Year|
+----------+------+----------+-----+----+----+
|   Douglas|  Male|  8/6/1993| null|null|null|
|    Thomas|  Male| 3/31/1996| null|null|null|
|     Maria|Female| 4/23/1993| null|null|null|
|     Jerry|  Male|  3/4/2005| null|null|null|
|     Larry|  Male| 1/24/1998| null|null|null|
|    Dennis|  Male| 4/18/1987| null|null|null|
|      Ruby|Female| 8/17/1987| null|null|null|
|      null|Female| 7/20/2015| null|null|null|
|    Angela|Female|11/22/2005| null|null|null|
|   Frances|Female|  8/8/2002| null|null|null|
|    Louise|Female| 8/12/1980| null|null|null|
|     Julie|Female|10/26/1997| null|null|null|
|   Brandon|  Male| 12/1/1980| null|null|null|
|      Gary|  Male| 1/27/2008| null|null|null|
|  Kimberly|Female| 1/14/1999| null|null|null|
|   Lillian|Female|  6/5/2016| null|null|null|
|    Jeremy|  Male| 9/21/2010| null|null|null|
|     Shawn|  Male| 12/7/1986| null|null|null|
|     Diana|F

In [76]:
total_df=res.unionByName(df5, allowMissingColumns=True)
total_df.show()

+----------+------+----------+-----+----+----+---------------+------+-------+-----------------+----+
|First Name|Gender|Start Date|Month| Day|Year|Last Login Time|Salary|Bonus %|Senior Management|Team|
+----------+------+----------+-----+----+----+---------------+------+-------+-----------------+----+
|   Douglas|  Male|  8/6/1993| null|null|null|           null|  null|   null|             null|null|
|    Thomas|  Male| 3/31/1996| null|null|null|           null|  null|   null|             null|null|
|     Maria|Female| 4/23/1993| null|null|null|           null|  null|   null|             null|null|
|     Jerry|  Male|  3/4/2005| null|null|null|           null|  null|   null|             null|null|
|     Larry|  Male| 1/24/1998| null|null|null|           null|  null|   null|             null|null|
|    Dennis|  Male| 4/18/1987| null|null|null|           null|  null|   null|             null|null|
|      Ruby|Female| 8/17/1987| null|null|null|           null|  null|   null|             n

AttributeError: ignored